## 2.1使用多个界定符分割字符串

### 关于正则表达式可以看这个链接[正则表达式](https://deerchao.net/tutorials/regex/regex.htm#escape)

在Python的string前面加上‘r’， 是为了告诉编译器这个string是个raw string，不要转意backslash '\' 。 例如，\n 在raw string中，是两个字符，\和n， 而不会转意为换行符。由于正则表达式和 \ 会有冲突，因此，当一个字符串使用了正则表达式后，最好在前面加上'r'。

In [12]:
# 保留分割字符 重新构造原来的字符串
import re
line = 'asdf fjdk; afed, fjek,asdf, foo'
# python中()叫捕获分组 其中匹配的文本也会出现在结果列表中
fileds = re.split(r'(;|,|\s)\s*',line)
fileds

['asdf', ' ', 'fjdk', ';', 'afed', ',', 'fjek', ',', 'asdf', ',', 'foo']

In [10]:
fileds = re.split(r'[;,\s]\s*',line)
fileds

['asdf', 'fjdk', 'afed', 'fjek', 'asdf', 'foo']

In [13]:
values = fileds[::2]
delimiters = fileds[1::2]

In [14]:
''.join(v+d for v,d in zip(values,delimiters))

'asdf fjdk;afed,fjek,asdf,'

## 2.2字符串开头或结尾匹配

In [ ]:
检查字符串开头或结尾的一个简单方法是使用 str.startswith() 或者是 str.endswith() 方法。

In [19]:
import os
filename = os.listdir('.')
filename

['chapter2.ipynb', 'chapter 1.ipynb', '.ipynb_checkpoints']

In [25]:
# python any() 如果都为空、0、false，则返回false，如果不都为空、0、false，则返回true。
# python all() 如果iterable的所有元素不为0、''、False或者iterable为空，all(iterable)返回True，否则返回False；

In [33]:
print(any(name.endswith('.ipynb') for name in filename))
print(all(name.endswith('.ipynb') for name in filename))

True
False


但是这两个方法只能转入元组或`str`作为参数 `list`或`set`都会报错，必须先转化为元组或`str`

In [ ]:
choice = ['http']
url = 'http://www.baidu.com'
try: 
    url.startswith(choice)
except:
    print("错误")

## 2.3 用Shell通配符匹配字符串

使用`Unix Shell`中的通配符如（`*.py`,`Dat[0-9]\*.csv`)，用`fnmatch`模块中的两个函数 `fnmathch()`和`fnamatchcase()`  
其中`fnmatch()`的大小写敏感规则和所用的操作系统一样
'fnamatchcase()'的完全根据提供的大小写匹配

In [54]:
from fnmatch import fnmatch, fnmatchcase
print(fnmatch('foo.txt','*.TXT')) # mac false windows true
print(fnmatchcase('foo.txt','*.txt'))
# 比较复杂的还是使用正则表达式

False
True


In [71]:
text = '11/27/2012'
if re.match(r'[1-9]+/\d+/\d',text):
    print("y")
else:
    print('n')

y


## 2.4 字符串匹配和搜索

如果你想使用同一个模式去做多次匹配，你应该先将模式字符串预编译为模式对象。使用`re.compile()`

In [76]:
import re
text1 = '11/27/2012 tomorrow is 11/28/2012'
text2 = 'nov 27, 2012'
datepat = re.compile(r'\d+/\d+/\d+')
def comparet(text):
    if datepat.match(text):
        print('yes')
    else:
        print('no')
comparet(text1)
comparet(text2)

yes
no


`match()`是从字符串开始去匹配，查找字符串任意部分的模式出现位置，使用`findall()`

In [77]:
datepat.findall(text1)

['11/27/2012', '11/28/2012']

若使用`()`去捕获分组，后面可以分别将每个分组的内容提取出来

In [96]:
datepat = re.compile(r'(\d+)/(\d+)/(\d+)')
m = datepat.match(text1)
print(m.group(0))
print(m.group(1))
print(m.group(2))
print(m.group(3))
print(m.groups())
month, day, year = m.groups()


11/27/2012
11
27
2012
('11', '27', '2012')


`findall()` 方法会搜索文本并以列表形式返回所有的匹配。 如果你想以迭代方式返回匹配，可以使用 `finditer()` 方法来代替

In [ ]:
for m in datepat.finditer(text):
    print(m.groups())

值得注意的是`re.match`仅仅检查字符串开始部分不能精确匹配

In [105]:
if datepat.match('11/27/2012abcdef'):
    print('y')
else:
    print('n')

if re.match(r'\d+/\d+/\d+$','11/27/2012abcdef'):
    print('y')
else:
    print('n')

y
n


## 2.5字符串搜索和替换

可以使用`re`中的`re.sub()`模块进行替换，第一个参数是被匹配的模式，第二个参数是替换模式。反斜杠数字比如`\3`指向前面模式的捕获组号。

In [107]:
text = 'Today is 5/15/2019. pyCon starts 5/13/2019'
import re 
re.sub(r'(\d+)/(\d+)/(\d+)',r'\3-\1-\2',text)
datepat = re.compile(r'(\d+)/(\d+)/(\d+)')
datepat.sub(r'\3-\1-\2',text)

'Today is 2019-5-15. pyCon starts 2019-5-13'

复杂的替换可以传递一个替换回掉函数来代替，一个替换回掉函数的参数是`match()`或者`find()`返回的对象，使用`group()`方法提取特定的匹配部分,回掉函数最后返回替换字符串

In [115]:
from calendar import month_abbr
print(month_abbr[1]) # 这个的作用是输入月份返回英文
def change_date(m):
    mon_name = month_abbr[int(m.group(1))]
    return '{} {} {}'.format(m.group(2), mon_name, m.group(3))
datepat.sub(change_date,text)

Jan


'Today is 15 May 2019. pyCon starts 13 May 2019'

`re.subn()`可以查看替换了多少

In [119]:
newtext, n = datepat.subn(r'\3-\1-\2', text)
print(newtext)
n

Today is 2019-5-15. pyCon starts 2019-5-13


2

## 2.6 字符串忽略大小写的搜索替换

给`re`模块提供`re.IGNORECASE`标志参数，忽略大小写

In [120]:
text = 'UPPER PYTHON, lower python, Mixed Python'
re.sub('python', 'snake', text, flags=re.IGNORECASE)

'UPPER snake, lower snake, Mixed snake'

最后的那个例子揭示了一个小缺陷，替换字符串并不会自动跟被匹配字符串的大小写保持一致。 为了修复这个，你可能需要一个辅助函数，就像下面的这样：

In [121]:
def matchcase(word):
    def replace(m):
        text = m.group()
        if text.isupper():
            return word.upper()
        elif text.islower():
            return word.lower()
        elif text[0].isupper():
            return word.capitalize()
        else:
            return word
    return replace

In [122]:
re.sub('python', matchcase('snake'), text, flags=re.IGNORECASE)

'UPPER SNAKE, lower snake, Mixed Snake'

## 2.7最短匹配模式

一般的匹配模式都是贪婪匹配，即尽可能匹配多的内容，尤其是类似`.*`的操作符，会尽可能多的匹配文本，在`*`操作符后面加上`?`就可以把贪婪匹配转化为非贪婪模式

In [156]:
str_pat = re.compile(r'"(.*)"')
text1 = 'Computer says "no." Phone says "yes."'
print(str_pat.findall(text1))
print(re.findall(r'"(.*?)"',text1))

['no." Phone says "yes.']
['no.', 'yes.']
